In [1]:
# Cell 1: Install all required packages
!pip install openai-whisper fastapi uvicorn python-multipart nest-asyncio pyngrok --quiet
!apt-get install -y ffmpeg --quiet
print("✅ All packages installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
✅ All packages installed!


In [2]:
# Cell 2: Import all libraries
import whisper
import os
import subprocess
import json
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import JSONResponse
from pyngrok import ngrok
import threading
import time

nest_asyncio.apply()
print("✅ Libraries imported!")

✅ Libraries imported!


In [3]:
# Cell 3: Load the Whisper model (run this once — takes a minute)
# Options: "tiny", "base", "small", "medium", "large"
# For laptop testing: "base" or "small" is recommended (fast + decent accuracy)
# For best accuracy: "medium" or "large" (slower)

MODEL_SIZE = "base"  # Change this if needed

print(f"⏳ Loading Whisper model: '{MODEL_SIZE}'...")
model = whisper.load_model(MODEL_SIZE)
print(f"✅ Whisper model '{MODEL_SIZE}' loaded and ready!")

⏳ Loading Whisper model: 'base'...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 102MiB/s]


✅ Whisper model 'base' loaded and ready!


In [4]:
# Cell 4: Define the FastAPI app with /transcribe endpoint

app = FastAPI(title="Whisper Transcription Service")

@app.get("/")
def root():
    return {"status": "Whisper service is running!", "model": MODEL_SIZE}

@app.post("/transcribe")
async def transcribe_video(
    video: UploadFile = File(...),
    job_id: str = Form(...)
):
    video_path = f"/tmp/{job_id}_video.mp4"
    audio_path = f"/tmp/{job_id}_audio.wav"

    try:
        print(f"\n📥 Received job_id: {job_id} | File: {video.filename}")

        # Step 1: Save uploaded video to disk
        with open(video_path, "wb") as f:
            content = await video.read()
            f.write(content)
        print(f"✅ Video saved: {video_path}")

        # Step 2: Extract audio using ffmpeg (mono, 16kHz WAV — ideal for Whisper)
        ffmpeg_cmd = [
            "ffmpeg", "-y",           # -y = overwrite output if exists
            "-i", video_path,
            "-vn",                    # no video
            "-acodec", "pcm_s16le",   # 16-bit PCM
            "-ar", "16000",           # 16kHz sample rate
            "-ac", "1",               # mono channel
            audio_path
        ]
        result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True)

        if result.returncode != 0:
            raise Exception(f"ffmpeg error: {result.stderr}")
        print(f"✅ Audio extracted: {audio_path}")

        # Step 3: Run Whisper transcription
        print("⏳ Transcribing... (this may take a while for long videos)")
        transcription = model.transcribe(audio_path, verbose=False)
        print("✅ Transcription complete!")

        # Step 4: Format response to match Person 4's expected structure
        duration = transcription["segments"][-1]["end"] if transcription["segments"] else 0.0

        formatted_segments = [
            {
                "id": seg["id"],
                "start": round(seg["start"], 2),
                "end": round(seg["end"], 2),
                "text": seg["text"].strip(),
                "confidence": round(1.0 - seg.get("no_speech_prob", 0.0), 2)
            }
            for seg in transcription["segments"]
        ]

        response = {
            "text": transcription["text"].strip(),
            "segments": formatted_segments,
            "language": transcription["language"],
            "duration": round(duration, 2)
        }

        print(f"📊 Language: {response['language']} | Duration: {response['duration']}s | Segments: {len(formatted_segments)}")
        return JSONResponse(content=response, status_code=200)

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return JSONResponse(content={"error": str(e)}, status_code=500)

    finally:
        # Step 5: Cleanup temp files
        for path in [video_path, audio_path]:
            if os.path.exists(path):
                os.remove(path)
                print(f"🧹 Cleaned up: {path}")

print("✅ FastAPI app defined!")

✅ FastAPI app defined!


In [6]:
# Cell 5: Set up ngrok and expose the service publicly
# Get your free auth token from: https://dashboard.ngrok.com/get-started/your-authtoken

NGROK_AUTH_TOKEN = "39vUgthPfzH3NGuzDlrPeGgCZ1R_4BTj1TFMMqAEHG3xENwaH"  # ← Paste your token here

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Kill any existing tunnels
ngrok.kill()

# Start tunnel on port 8000
public_url = ngrok.connect(8000)
print("=" * 50)
print(f"🌐 Whisper Service PUBLIC URL:")
print(f"   {public_url}")
print("=" * 50)
print(f"\n📋 Tell Person 4 to set in .env:")
print(f'   WHISPER_SERVICE_URL={public_url}')
print(f"\n📋 Transcribe endpoint:")
print(f'   POST {public_url}/transcribe')

🌐 Whisper Service PUBLIC URL:
   NgrokTunnel: "https://semioratorical-imputrescible-kristel.ngrok-free.dev" -> "http://localhost:8000"

📋 Tell Person 4 to set in .env:
   WHISPER_SERVICE_URL=NgrokTunnel: "https://semioratorical-imputrescible-kristel.ngrok-free.dev" -> "http://localhost:8000"

📋 Transcribe endpoint:
   POST NgrokTunnel: "https://semioratorical-imputrescible-kristel.ngrok-free.dev" -> "http://localhost:8000"/transcribe


In [11]:
# Cell 6 : Run uvicorn in a background thread (Colab-compatible)
import asyncio
import threading

config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

# Run server in a separate thread so Colab's event loop doesn't conflict
thread = threading.Thread(target=server.run, daemon=True)
thread.start()

# Wait a moment for server to start
time.sleep(2)
print("🚀 Whisper server is running on port 8000!")
print("✅ Service is ready to accept requests!")
print("⚠️  Keep this cell's session alive (don't restart runtime)")

INFO:     Started server process [325]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


🚀 Whisper server is running on port 8000!
✅ Service is ready to accept requests!
⚠️  Keep this cell's session alive (don't restart runtime)


In [17]:
# Cell 7: Test your service — run this from your laptop or another Colab cell
# Replace the URL with your actual ngrok URL from Cell 5

import requests

WHISPER_URL = "https://semioratorical-imputrescible-kristel.ngrok-free.dev"  # ← Paste your ngrok URL

# Test 1: Health check
print("🔍 Testing health check...")
response = requests.get(f"{WHISPER_URL}/")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

# Test 2: Transcription (replace with your video file path)
print("\n🎥 Testing transcription...")
VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/Creative Writing for Kids  7 Tips - Mr Tim's Classroom (1080p, h264) (1).mp4"  # ← Path to a test video on your laptop

with open(VIDEO_PATH, "rb") as f:
    files = {"video": (os.path.basename(VIDEO_PATH), f, "video/mp4")}
    data = {"job_id": "test_001"}
    response = requests.post(
        f"{WHISPER_URL}/transcribe",
        files=files,
        data=data,
        timeout=300  # 5 minute timeout
    )

print(f"Status: {response.status_code}")
result = response.json()

if "error" in result:
    print(f"❌ Error: {result['error']}")
else:
    print(f"✅ Language: {result['language']}")
    print(f"✅ Duration: {result['duration']}s")
    print(f"✅ Segments: {len(result['segments'])}")
    print(f"\n📝 First 3 segments:")
    for seg in result["segments"][:3]:
        print(f"  [{seg['start']}s → {seg['end']}s] {seg['text']}")
    print(f"\n📝 Full transcript preview:")
    print(result["text"][:500] + "..." if len(result["text"]) > 500 else result["text"])

🔍 Testing health check...
INFO:     136.117.128.91:0 - "GET / HTTP/1.1" 200 OK
Status: 200
Response: {'status': 'Whisper service is running!', 'model': 'base'}

🎥 Testing transcription...

📥 Received job_id: test_001 | File: Creative Writing for Kids  7 Tips - Mr Tim's Classroom (1080p, h264) (1).mp4
✅ Video saved: /tmp/test_001_video.mp4
✅ Audio extracted: /tmp/test_001_audio.wav
⏳ Transcribing... (this may take a while for long videos)
Detected language: English


100%|██████████| 75933/75933 [00:26<00:00, 2880.69frames/s]

✅ Transcription complete!
📊 Language: en | Duration: 753.0s | Segments: 153
🧹 Cleaned up: /tmp/test_001_video.mp4
🧹 Cleaned up: /tmp/test_001_audio.wav
INFO:     136.117.128.91:0 - "POST /transcribe HTTP/1.1" 200 OK
Status: 200
✅ Language: en
✅ Duration: 753.0s
✅ Segments: 153

📝 First 3 segments:
  [0.0s → 28.48s] A
  [30.0s → 37.0s] Hi kids welcome back to Mr. Tim's classroom. I know it's been a very challenging time for you.
  [37.0s → 44.0s] No doubt you've been in front of screens a lot. I just hope that your eyes aren't turning square.

📝 Full transcript preview:
A Hi kids welcome back to Mr. Tim's classroom. I know it's been a very challenging time for you. No doubt you've been in front of screens a lot. I just hope that your eyes aren't turning square. I just thought today I might share a little video on creative writing. So I want to give you seven tips to really improve your creative writing skills. Shall we do it together? Alright kids my first tip is start with excitement. 